In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("BHP.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print(df[col].value_counts())
    print("*"*20)

In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns=['area_type','availability','society','balcony'],inplace=True)

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# filling up the missing value in location
missing_value  = df[df['location'].isnull()]
print(missing_value)

In [ ]:
df['location'] = df['location'].fillna('Sarjapur  Road')

In [ ]:
df['location'].isna().sum()

In [ ]:
# Filling up missing values in Size
missing_value  = df[df['size'].isnull()]
print(missing_value)

In [ ]:
df['size'] = df['size'].fillna('2 BHK')

In [ ]:
df['size'].isna().sum()

In [ ]:
#  Filling missing values in Bath
missing_value  = df[df['bath'].isnull()]
print(missing_value)

In [ ]:
df['bath'] = df['bath'].fillna(df['bath'].median())

In [ ]:
df['bath'].isna().sum()

In [ ]:
df.info()

In [ ]:
#  fixing the BHK column
df['bhk'] = df['size'].str.split().str.get(0).astype(int)

In [ ]:
df.head()

In [ ]:
#  Fixing the total_sqft column
df['total_sqft'].unique()

In [ ]:
def convert_range(x):
    
    temp = x.split("-")
    if len(temp) == 2:
        return (float(temp[0]) + float(temp[1]))/2
    
    try:
        return float(x)
    except:
        return None

In [ ]:
df['total_sqft'] = df['total_sqft'].apply(convert_range)

In [ ]:
df['total_sqft'].unique()

In [ ]:
# Create a new column price per sqft
df['price_per_sqft'] = df['price']*100000 / df['total_sqft']

In [ ]:
df.describe()

In [ ]:
#  reducing the value_count in location
df['location'] = df['location'].apply(lambda x:x.strip())
location_count = df['location'].value_counts()

In [ ]:
location_count_under_10 = location_count[location_count <= 10]
location_count_under_10

In [ ]:
df['location'] = df['location'].apply(lambda x: 'other' if x in location_count_under_10 else x)

In [ ]:
df['location'].value_counts()

In [ ]:
#  Removing outliers:

df = df[((df['total_sqft']/df['bhk']) >= 300)]
df.describe()

In [ ]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        gen_df = subdf[(subdf.price_per_sqft > (m - st)) & (subdf.price_per_sqft <= (m + st)) ]
        df_output = pd.concat([df_output,gen_df],ignore_index = True)
    return df_output
df = remove_outliers_sqft(df)
df.describe()

In [ ]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] ={
                'mean' : np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count' : bhk_df.shape[0]
            }
        for bhk, bhk_stats in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk - 1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft < (stats['mean'])].index.values)
    return df.drop(exclude_indices, axis='index')

In [ ]:
df = bhk_outlier_remover(df)

In [ ]:
df.shape

In [ ]:
df.drop(columns=['size','price_per_sqft'],inplace=True)

In [ ]:
df

In [ ]:
df.to_csv('Cleaned_data.csv')

In [ ]:
X = df.drop(columns = ['price'])
y = df['price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =2)

In [ ]:
print(X_train.shape)
print(X_test.shape)

Applying Linear Regression


In [ ]:
column_trans = make_column_transformer((OneHotEncoder(sparse = False),['location']), remainder = 'passthrough')
scaler = StandardScaler()
lr = LinearRegression()

In [ ]:
pipe = make_pipeline(column_trans, scaler, lr)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred_lr = pipe.predict(X_test)

In [ ]:
r2_score(y_test,y_pred_lr)

Lasso Regression


In [ ]:
lasso = Lasso()

In [ ]:
pipe = make_pipeline(column_trans, scaler, lasso)

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred_lasso = pipe.predict(X_test)

In [ ]:
r2_score(y_test,y_pred_lasso)

Ridge Regression


In [ ]:
ridge = Ridge()

In [ ]:
pipe = make_pipeline(column_trans, scaler, ridge)

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred_ridge = pipe.predict(X_test)

In [ ]:
r2_score(y_test, y_pred_ridge)

In [ ]:
import pickle

In [ ]:
pickle.dump(pipe, open('RidgeModel.pkl','wb'))

In [ ]:
pipe = pickle.load(open('RidgeModel.pkl', 'rb'))

# Prepare input data with correct columns
input_data = pd.DataFrame([{
    'location': '1st Phase JP Nagar',
    'total_sqft': 1500,
    'bath': 2,
    'bhk': 2
}])

# Predict
result = pipe.predict(input_data)


In [ ]:
result

In [ ]:
import sklearn
print(sklearn.__version__) 

In [ ]:
!pip install --upgrade scikit-learn


In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn
